# Challenge 7

### Fit to Gaussian peak in 2D with unbinned likelihood Poissonian

Our detector now keeps track of counts in a (x, y) plane. Let's print our data:

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
#from numba import jit

np.random.seed(
    0
)  # We manually set the seed for reproducible results, this is not required

pos_x = np.array([-9.7, -9.3, -6.9, -6.7, -5.7, -5.1, -2.5, -2.5, -2.3, -2.3, -1.9, -1.9, 
                  -1.9, -1.7, -0.7, -0.7, -0.3, -0.1, 0.1, 0.1, 0.3, 0.5, 0.9, 0.9, 0.9,
                  1.3, 1.7, 2.5, 3.3, 4.7, 5.1, 5.5, 5.9, 7.3, 8.3, 8.7, 8.9, 9.1, 9.5, 9.7])

pos_y = np.array([-2.9, -6.1, 9.5, -3.7, -8.9, -9.5, 0.1, 3.3, -2.9, -1.7, 0.1, 0.3, 7.5, 
                 1.9, -1.5, 0.1, -5.9, 6.9, -0.3, 2.1, 2.9, -1.7, -1.5, -0.3, 0.5, -3.3, 
                 6.3, 0.9, -9.3, -4.1, -9.3, -5.7, 7.5, 5.3, -8.7, 0.5, 2.7, 4.9, -9.5, 8.5])

N = len(pos_x) # Total number of events

print("Count coordinates: ")
for i in range(N):
    print(f"({pos_x[i]}, {pos_y[i]}) ")

Count coordinates: 
(-9.7, -2.9) 
(-9.3, -6.1) 
(-6.9, 9.5) 
(-6.7, -3.7) 
(-5.7, -8.9) 
(-5.1, -9.5) 
(-2.5, 0.1) 
(-2.5, 3.3) 
(-2.3, -2.9) 
(-2.3, -1.7) 
(-1.9, 0.1) 
(-1.9, 0.3) 
(-1.9, 7.5) 
(-1.7, 1.9) 
(-0.7, -1.5) 
(-0.7, 0.1) 
(-0.3, -5.9) 
(-0.1, 6.9) 
(0.1, -0.3) 
(0.1, 2.1) 
(0.3, 2.9) 
(0.5, -1.7) 
(0.9, -1.5) 
(0.9, -0.3) 
(0.9, 0.5) 
(1.3, -3.3) 
(1.7, 6.3) 
(2.5, 0.9) 
(3.3, -9.3) 
(4.7, -4.1) 
(5.1, -9.3) 
(5.5, -5.7) 
(5.9, 7.5) 
(7.3, 5.3) 
(8.3, -8.7) 
(8.7, 0.5) 
(8.9, 2.7) 
(9.1, 4.9) 
(9.5, -9.5) 
(9.7, 8.5) 


We use numba to speed up the execution

In [2]:
@jit
def model(E, S, B, E_0=4, sigma_E=1):
    return B + S * np.exp(-((E - E_0) ** 2) / (2 * sigma_E ** 2))

NameError: name 'jit' is not defined

In [ ]:
@jit
def log_likelihood(bin_centers, counts, S, B):
    result = 0
    for i in range(len(counts)):
        # Poisson bins
        mu = model(bin_centers[i], S=S, B=B)
        result += 2 * (mu - counts[i] + counts[i] * np.log(counts[i] / mu))
    return result

## Fit data

In [ ]:
@jit
def minimize_loglike(bins, counts, N=100):
    S_result = 0
    B_result = 0

    log_min = 1e10

    for S in np.linspace(0.01, 20, N):
        for B in np.linspace(0, 20, N):
            loglike = log_likelihood(energies, counts, S=S, B=B)
            if loglike < log_min:
                log_min = loglike
                S_result = S
                B_result = B

    return log_min, S_result, B_result


log_min_ref, S_result_ref, B_result_ref = minimize_loglike(energies, counts, N=500)

print(f"S={S_result_ref:0.3f}, B={B_result_ref:0.3f}, logmin={log_min_ref:0.3f}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

ax.set_xlim([0, 10])

ax.set_xlabel("Energy (a.u.)")
ax.set_ylabel("Counts (a.u.)")
ax.set_title("Measured spectrum")

ax.errorbar(energies, counts, xerr=0.5, fmt="o", yerr=np.sqrt(counts), color="blue")

ax.bar(energies, counts, color="blue", alpha=0.5)

x = np.linspace(0, 10, 200)
ax.plot(x, model(x, S=S_result_ref, B=B_result_ref), color="red", linewidth=3)

plt.show()

## MC study to study distribution

In [ ]:
import sys

number_of_samples = 10000

log_min_dist = np.zeros(number_of_samples)
S_dist = np.zeros(number_of_samples)
B_dist = np.zeros(number_of_samples)

for i in range(number_of_samples):
    if i % int(number_of_samples / 10) == 0:
        print(str(i / number_of_samples * 100) + "%")
    sys.stdout.flush()  # to avoid warning due to numba

    sample = [model(energy, S=S_result_ref, B=B_result_ref) for energy in energies]
    sample = np.array([st.poisson(int(count)).rvs() for count in sample])

    log_min, S_result_mc, B_result_mc = minimize_loglike(energies, sample)
    # print(log_min, S_result_mc, B_result_mc)
    log_min_dist[i] = log_min
    S_dist[i] = S_result_mc
    B_dist[i] = B_result_mc

In [ ]:
bins = np.linspace(0, 30, 50)

plt.hist(
    log_min_dist, bins=bins, color="blue", alpha=0.5, density=True, label="MC Data"
)
plt.plot(bins, st.chi2(df=10 - 2).pdf(bins), label="Chi2 (df=8)", color="red")
plt.axvline(x=log_min_ref, color="black", linewidth=3, label="fit")
plt.legend()

plt.xlabel("MaxLogLikelihood")
plt.show()

print(f"p-value = {1 - st.chi2(df=10 - 2).cdf(log_min_ref):0.2f}")

## Coverage of the confidence intervals on S_hat and B_hat

In [ ]:
mean, sigma = np.mean(B_dist), np.std(B_dist)
conf_int = st.norm.interval(0.68, loc=mean, scale=sigma)

print(f"Confidence interavl for B: {conf_int}")

mean, sigma = np.mean(S_dist), np.std(S_dist)
conf_int = st.norm.interval(0.68, loc=mean, scale=sigma)

print(f"Confidence interavl for S: {conf_int}")

In [ ]:
bound = 8.0
S_interval = []
B_interval = []

for i in range(len(log_min_dist)):
    if log_min_dist[i] < bound:
        S_interval.append(S_dist[i])
        B_interval.append(B_dist[i])

plt.scatter(S_interval, B_interval)
plt.scatter([S_result_ref], [B_result_ref], color="red")
plt.xlabel("S")
plt.ylabel("B")
plt.show()